In [1]:
import sqlite3
import pandas as pd
import joblib
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Conexión a la base de datos
conn = sqlite3.connect("db/market_place.db")

# Cargar recursos
productos = pd.read_sql_query("SELECT product_id, product_name FROM products", conn)
usuarios = pd.read_sql_query("SELECT user_id, segmento FROM users", conn)
rules = joblib.load("models/MBA/MBA_reglas_apriori.joblib")
tfidf_vectorizer = joblib.load("models/NLP/tfidf_vectorizer.joblib")
tfidf_matrix = joblib.load("models/NLP/tfidf_matrix.joblib")

# NearestNeighbors para NLP
nn_model = NearestNeighbors(n_neighbors=20, metric='cosine').fit(tfidf_matrix)

# Modelos por segmento
segmentos_modelos = {
    segmento: joblib.load(f"models/SVD/SVD_{segmento}.joblib")
    for segmento in usuarios['segmento'].unique()
}

# Obtener segmento
def get_segmento_usuario(user_id):
    row = usuarios[usuarios['user_id'] == user_id]
    return row.iloc[0]['segmento'] if not row.empty else None

# Recomendaciones precalculadas de SVD
def recomendaciones_svd(user_id, top_n=10):
    query = """
    SELECT product_id, score FROM svd_predictions
    WHERE user_id = ?
    ORDER BY score DESC
    LIMIT ?
    """
    df = pd.read_sql_query(query, conn, params=(user_id, top_n))
    return df['product_id'].tolist()

# MBA rápido
def recomendaciones_mba(productos_en_carrito):
    recomendados = set()
    for producto in productos_en_carrito:
        sub_rules = rules[rules['antecedents'].apply(lambda x: producto in x)]
        recomendados.update(sub_rules['consequents'].explode().tolist())
    return list(recomendados)

# NLP rápido con NearestNeighbors
def recomendaciones_nlp(query, top_n=10):
    vector = tfidf_vectorizer.transform([query])
    indices = nn_model.kneighbors(vector, return_distance=False)[0][:top_n]
    return productos.iloc[indices]['product_id'].tolist()

# Motor híbrido explicativo
def recomendar_para_usuario_explicado(user_id, productos_en_carrito=[], busqueda=None, top_n=10):
    rec_svd = recomendaciones_svd(user_id, top_n * 2)
    rec_mba = recomendaciones_mba(productos_en_carrito)
    rec_nlp = recomendaciones_nlp(busqueda, top_n * 2) if busqueda else []

    pesos = {'svd': 0.5, 'mba': 0.3, 'nlp': 0.2}
    explicacion = defaultdict(lambda: {'peso_total': 0, 'detalles': {}})

    # SVD
    for pid in rec_svd:
        score = conn.execute("SELECT score FROM svd_predictions WHERE user_id=? AND product_id=?", (user_id, pid)).fetchone()
        if score:
            explicacion[pid]['peso_total'] += pesos['svd']
            explicacion[pid]['detalles']['svd'] = round(score[0], 3)

    # MBA
    for pid in rec_mba:
        explicacion[pid]['peso_total'] += pesos['mba']
        explicacion[pid]['detalles']['mba'] = "Regla encontrada"

    # NLP
    if busqueda:
        vector = tfidf_vectorizer.transform([busqueda])
        indices = nn_model.kneighbors(vector, return_distance=False)[0][:top_n * 2]
        for idx in indices:
            pid = productos.iloc[idx]['product_id']
            explicacion[pid]['peso_total'] += pesos['nlp']
            explicacion[pid]['detalles']['nlp'] = "Coincidencia con búsqueda"

    # Armar resultados
    final = sorted(explicacion.items(), key=lambda x: x[1]['peso_total'], reverse=True)[:top_n]
    resultados = []
    for pid, data in final:
        nombre = productos[productos['product_id'] == pid]['product_name'].values[0]
        resultados.append({
            'product_id': pid,
            'product_name': nombre,
            'peso_total': round(data['peso_total'], 3),
            'detalles': data['detalles']
        })
    return resultados

# Recomendador contextual rápido
def recomendar_contextual(user_id, productos_en_carrito=None, busqueda=None, top_n=10):
    productos_en_carrito = productos_en_carrito or []
    hay_busqueda = bool(busqueda)
    hay_carrito = bool(productos_en_carrito)

    if not hay_busqueda and not hay_carrito:
        return recomendar_para_usuario_explicado(user_id, [], None, top_n)
    elif hay_busqueda and not hay_carrito:
        rec_nlp_ids = recomendaciones_nlp(busqueda, top_n)
        return [{
            'product_id': pid,
            'product_name': productos[productos['product_id'] == pid]['product_name'].values[0],
            'peso_total': 0.2,
            'detalles': {'nlp': 'Coincidencia con búsqueda'}
        } for pid in rec_nlp_ids]
    else:
        return recomendar_para_usuario_explicado(user_id, productos_en_carrito, busqueda, top_n)

c:\Users\joant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\joant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
user = 6786

print("\n📦 Inicio (sin carrito, sin búsqueda):")
recomendaciones = recomendar_contextual(user)
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Sugerencia basada en perfil)")

print("\n🔍 Búsqueda: 'cookie'")
recomendaciones = recomendar_contextual(user, busqueda="cookie")
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Coincidencia con búsqueda)")

print("\n🛒 Carrito contiene producto ID 22802 (chocolate chip cookies)")
recomendaciones = recomendar_contextual(user, productos_en_carrito=[22802], busqueda="cookie")
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Peso: {r['peso_total']})")
    for fuente, valor in r['detalles'].items():
        print(f"   - {fuente.upper()}: {valor}")



📦 Inicio (sin carrito, sin búsqueda):
👉 plain greek yogurt (Sugerencia basada en perfil)
👉 1 lowfat milk (Sugerencia basada en perfil)
👉 organic bread with 21 whole grains (Sugerencia basada en perfil)
👉 organic honey sweet whole wheat bread (Sugerencia basada en perfil)
👉 italian sparkling mineral water (Sugerencia basada en perfil)
👉 unsweetened original almond breeze almond milk (Sugerencia basada en perfil)
👉 organic milk reduced fat 2 milkfat (Sugerencia basada en perfil)
👉 organic whole string cheese (Sugerencia basada en perfil)
👉 michigan organic kale (Sugerencia basada en perfil)
👉 natural artesian water (Sugerencia basada en perfil)

🔍 Búsqueda: 'cookie'
👉 vanilla cookie (Coincidencia con búsqueda)
👉 cookie with coffee (Coincidencia con búsqueda)
👉 cookie sugar (Coincidencia con búsqueda)
👉 sugar cookie (Coincidencia con búsqueda)
👉 cookie bars (Coincidencia con búsqueda)
👉 ginger cookie (Coincidencia con búsqueda)
👉 cookie sticks (Coincidencia con búsqueda)
👉 the complete c